In [1]:
# As usual, a bit of setup

import numpy as np
import matplotlib.pyplot as plt
#from cs231n.classifiers.cnn import *
from cs231n.data_utils import get_CIFAR10_data
from cs231n.gradient_check import eval_numerical_gradient_array, eval_numerical_gradient
#from cs231n.layers import *
#from cs231n.fast_layers import *
#from cs231n.solver import Solver

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-re|loading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [ ]:
def limit_mem():
    import keras.backend as K
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

In [ ]:
#imit_mem()

In [2]:
data = get_CIFAR10_data()
for k, v in data.items():
  print('%s: ' % k, v.shape)

cs231n/datasets/cifar-10-batches-py/data_batch_1
cs231n/datasets/cifar-10-batches-py/data_batch_2
cs231n/datasets/cifar-10-batches-py/data_batch_3
cs231n/datasets/cifar-10-batches-py/data_batch_4
cs231n/datasets/cifar-10-batches-py/data_batch_5
X_train:  (49000, 3, 32, 32)
y_train:  (49000,)
X_val:  (1000, 3, 32, 32)
y_val:  (1000,)
X_test:  (1000, 3, 32, 32)
y_test:  (1000,)


In [3]:
X = data['X_train'][:1000]
y = data['y_train'][:1000]


In [ ]:
import tensorflow as tf

In [ ]:
#tf.reshape(X, [-1, 32, 32, ])

In [ ]:
from cs231n.classifiers.conv_net_tf import cnn_model, BATCH_SIZE, put_channel_last

 1. fc  -> softmax ->   backprop
 2.
 2. conv - relu - 2x2 max pool - affine - relu - affine - softmax

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
%%capture
tensors_to_log = {"total_loss": "total_loss"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)

!rm -rf tf_logs/
!mkdir tf_logs
tf.logging.set_verbosity(0)
clf = tf.estimator.Estimator(cnn_model, model_dir='tf_logs')
train_input_func = tf.estimator.inputs.numpy_input_fn(
    x={'x': put_channel_last(X)}, y=y,
    batch_size=BATCH_SIZE, shuffle=True, num_epochs=20)
tf.logging.set_verbosity(tf.logging.INFO)
clf.train(input_fn=train_input_func, steps=10, 
         hooks=[logging_hook]
         )

In [ ]:
#preds = clf.predict(train_input_func)

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
x={'x': put_channel_last(X)}, y=y, num_epochs=1,batch_size=BATCH_SIZE,
    shuffle=False)
eval_results= clf.evaluate(input_fn=eval_input_fn); eval_results

In [4]:
import torch


In [5]:
from cs231n.classifiers.conv_net import train_torchnet

In [ ]:
%%time
scores = {}
train_loss = {}
for filter_size in [1, 3, 5, 7, 9]:
    model = train_torchnet(data['X_train'], data['y_train'],
                             num_epochs=5, weight_decay=.001,
                             print_every=200,
                             X_val=data['X_val'], y_val=data['y_val'],
                             filter_size=filter_size,
                            use_cuda=True)
    scores[filter_size] = model.val_scores
    train_loss[filter_size] = model.train_loss

In [ ]:
%%time

score_dct = {}
loss_dct = {}
for weight_decay in [1e-4, 1e-3, 1e-2, 1e-1]:
    model = train_torchnet(data['X_train'], data['y_train'],
                             num_epochs=3,
                            #n_batches=400,
                            weight_decay=weight_decay,
                             print_every=200,
                             X_val=data['X_val'], y_val=data['y_val'],
                             filter_size=5,
                            use_cuda=True)
    
    score_dct[weight_decay] = model.val_scores.copy()
    loss_dct[weight_decay] = model.train_loss.copy()
    del model
    print(score_dct)

In [21]:
%%time
modelavg = train_torchnet(data['X_train'], data['y_train'],
                             num_epochs=20,
                             # n_batches=10,
                          batch_size=100,
                              num_filters=64,
                             weight_decay=.001,
                             print_every=500,
                             X_val=data['X_val'], y_val=data['y_val'],
                             filter_size=3,
                          padding=1,
                             #num_filters=64,
                             use_cuda=True)

  0%|          | 0/20 [00:00<?, ?it/s]

64
64 4096


  5%|▌         | 1/20 [00:07<02:13,  7.02s/it]

0.696


 10%|█         | 2/20 [00:14<02:06,  7.03s/it]

...

0.799


 95%|█████████▌| 19/20 [02:13<00:07,  7.05s/it]

0.81


100%|██████████| 20/20 [02:20<00:00,  7.05s/it]

CPU times: user 2min 20s, sys: 396 ms, total: 2min 20s
Wall time: 2min 20s


In [6]:
#from cs231n.classifiers.conv_net import ThreeLayerTorchNet

In [7]:
from torchvision.models.vgg import vgg11

In [8]:
#m = ThreeLayerTorchNet(num_filters=64, filter_size=3, padding=1)

In [ ]:
m.output_dim2

In [ ]:
64*128

In [ ]:
from torch.autograd import Variable
from torch import FloatTensor
m.forward(Variable(FloatTensor(X[:10])))

In [ ]:
?

In [ ]:
vgg

In [ ]:
vgg = vgg11(pretrained=True)

In [ ]:
model

In [ ]:
model3 = train_torchnet(data['X_train'], data['y_train'],
                             num_epochs=5,
                             # n_batches=10,
                             weight_decay=.001,
                             print_every=500,
                             X_val=data['X_val'], y_val=data['y_val'],
                             filter_size=3,
                             padding=1,
                             num_filters=64,
                             use_cuda=True)

In [ ]:
#model.train_loss

In [ ]:
%%time
#model.pool_height=2
dct = modelavg.custom_scorer(data); print(dct)

In [ ]:
modelavg.val_scores

In [ ]:
dct

In [ ]:
#train_score

In [ ]:
val_score

In [ ]:
pd.Series(model.val_scores).plot(style='-o')



In [ ]:
modeldo.train_loss

In [ ]:
pd.Series(model.train_loss).plot(style='-o')

In [ ]:
pd.DataFrame(score_dct).plot(style='-o')

In [ ]:
pd.DataFrame(loss_dct).plot(style='-o')

In [ ]:
pd.DataFrame(loss_dct)

In [ ]:
model = train_torchnet(data['X_train'], data['y_train'],
                         num_epochs=5, 
                       weight_decay=weight_decay,
                         print_every=200,
                         X_val=data['X_val'], y_val=data['y_val'],
                         filter_size=5,
                        use_cuda=True)

In [ ]:
#pd.DataFrame(loss_dct)

In [ ]:
model.val_scores

In [ ]:
torch_model.val_scores

In [ ]:
import pandas as pd
pd.Series(
    torch_model.val_scores
).plot(style='-o')

In [ ]:
torch_model.val_scores

In [ ]:
%%time
torch_model.custom_scorer(data)

In [ ]:
X.shape

In [ ]:
#for x,y in zip( batch(X[:30], n=10), batch(y[:30], n=10)):
    print(x,y)

In [ ]:
from cs231n.classifiers.conv_net import batch, train_torchnet

In [ ]:
torch_model.funcy_scorer(X, y)

In [ ]:
%%time
torch_model = train_torchnet(data['X_train'], data['y_train'],
                             n_batches=1000,
                             weight_decay=.001,
                            use_cuda=True)
print(
    torch_model.funcy_scorer(data['X_val'], data['y_val'])
)

In [ ]:
%%time
print(
    torch_model.funcy_scorer(data['X_train'], data['y_train'])
)

In [ ]:
%%time
torch_model = train_torchnet(data['X_train'], data['y_train'],
                             n_batches=10,
                             weight_decay=0,
                            use_cuda=False)
print(
    torch_model.funcy_scorer(data['X_val'], data['y_val'])
)

In [ ]:
torch_model

In [ ]:
%%time


In [ ]:
torch_model

In [ ]:
torch_model = train_torchnet(X, y, n_batches=10, weight_decay=1e-2)
torch_model.funcy_scorer(X, y)

In [ ]:
!pwd

In [ ]:
preds = clf.predict(input_fn=eval_input_fn)

In [ ]:
tf.metrics.accuracy(y, ['classes'])

In [ ]:
plist = list(preds)

In [ ]:
import pandas as pd

In [ ]:
y

In [ ]:
#pd.DataFrame(plist).assign(y=y)

In [ ]:
X[0].shape